In [13]:
# Load in our libraries
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re


import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from collections import Counter

# Feature Exploration, Engineering and Cleaning 

Now we will proceed much like how most kernels in general are structured, and that is to first explore the data on hand, identify possible feature engineering opportunities and one hot encode any categorical features.

In [14]:
# Load in the train and test datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Let's check if we have empty values

In [15]:
# Merge the train and test datasets
full_data = [train, test]
train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [16]:
print (train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


Age / Cabin / Embarked have empty values

# Feature Engineering #

## 1. Pclass ##
there is no missing value on this feature and already a numerical value. so let's check it's impact on our train set.

In [17]:
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


## 2. Sex ##

In [18]:
print (train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


## 3. SibSp and Parch ##
With the number of siblings/spouse and the number of children/parents we can create new feature called Family Size.

In [19]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
print (train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


## 3. Generate new feature - Is Alone ##

In [20]:
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538


## 4. Embarked ##
the embarked feature has some missing value. and we try to fill those with the most occurred value ( 'S' ).

In [21]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


## 5. Fare ##
Fare also has some missing value and we will replace it with the median. then we categorize it into 4 ranges

In [22]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
print (train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())

   CategoricalFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


## 6. Age ##
we have plenty of missing values in this feature. # generate random numbers between (mean - std) and (mean + std).
then we categorize age into 5 range.

In [23]:
for dataset in full_data:
    age_avg 	   = dataset['Age'].mean()
    age_std 	   = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
train['CategoricalAge'] = pd.cut(train['Age'], 5)

print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())

  CategoricalAge  Survived
0  (-0.08, 16.0]  0.513274
1   (16.0, 32.0]  0.360092
2   (32.0, 48.0]  0.366412
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


## 7. Name ##
inside this feature we can find the title of people.

In [24]:
def get_title(name):
	title_search = re.search(' ([A-Za-z]+)\.', name)
	# If the title exists, extract and return it.
	if title_search:
		return title_search.group(1)
	return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

print(pd.crosstab(train['Title'], train['Sex']))

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1


 so we have titles. let's categorize it and check the title impact on survival rate.

In [25]:
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

print (train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean())

    Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


# Data Cleaning #
great! now let's clean our data and map our features into numerical values.

In [26]:
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
#     dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
#     dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
#     dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
#     dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
#     dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
#     dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
#     dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
#     dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
#     dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4

# Feature Selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp',\
                 'Parch']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

test  = test.drop(drop_elements, axis = 1)

print (train.head(10))

   Survived  Pclass  Sex  Age     Fare  Embarked  FamilySize  IsAlone  Title
0         0       3    1   22   7.2500         0           2        0      1
1         1       1    0   38  71.2833         1           2        0      3
2         1       3    0   26   7.9250         0           1        1      2
3         1       1    0   35  53.1000         0           2        0      3
4         0       3    1   35   8.0500         0           1        1      1
5         0       3    1   33   8.4583         2           1        1      1
6         0       1    1   54  51.8625         0           1        1      1
7         0       3    1    2  21.0750         0           5        0      4
8         1       3    0   27  11.1333         0           3        0      3
9         1       2    0   14  30.0708         1           2        0      3


In [27]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['Survived']
train = train.drop(['Survived'], axis=1)
x_train = train.values
x_test = test.values

In [28]:
cat_columns = [u'Pclass', u'Title', u'Embarked', u'FamilySize']
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset_preprocessed = pd.get_dummies(dataset, prefix=cat_columns, columns=cat_columns)
x_train = dataset_preprocessed[:train_objs_num]
x_test = dataset_preprocessed[train_objs_num:]

Let's try with a Machine Learning Model - Random Forest Classifier

In [29]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None, params_needed=True):
        if params_needed:
            params['random_state'] = seed
            self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [30]:
def get_oof(clf, x_train, y_train, x_test, scaling = True):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    total_accuracy = 0.0
    
    if scaling:
        scaler = StandardScaler()
        columns = [u'Fare', u'Age']
        x_train[columns] = scaler.fit_transform(x_train[columns])
        x_test[columns] = scaler.transform(x_test[columns])

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train.as_matrix()[train_index]
        y_tr = y_train.as_matrix()[train_index]
        x_te = x_train.as_matrix()[test_index]
        y_te = y_train.as_matrix()[test_index]

        

        clf.train(x_tr, y_tr)

        predictions = clf.predict(x_te)
        score = accuracy_score(y_te, predictions)
        print(score)
        total_accuracy += score
        oof_train[test_index] = predictions
        oof_test_skf[i, :] = clf.predict(x_test)


    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1), total_accuracy / NFOLDS

Computing Accuracy of RF classifier

In [31]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params={})

In [32]:
rf_oof_train, rf_oof_test, total_accuracy = get_oof(rf,x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.810055865922
0.786516853933
0.814606741573
0.775280898876
0.786516853933
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.7945954428472789)


Computing accuracy of ADA Boost Classifier

In [33]:
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params={})
ada_oof_train, ada_oof_test, total_accuracy = get_oof(ada,x_train, y_train, x_test)
print("Adaboost classifier - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.787709497207
0.808988764045
0.808988764045
0.797752808989
0.848314606742
('Adaboost classifier - Total Accuracy with data cleanup and without one hot encoding', 0.8103508882053857)


We can try one-hot encoding the categorical variables

In [34]:
def get_oof_with_pca(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    total_accuracy = 0.0
    
    scaler = StandardScaler()
    columns = [u'Fare', u'Age']
    x_train[columns] = scaler.fit_transform(x_train[columns])
    x_test[columns] = scaler.transform(x_test[columns])

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train.as_matrix()[train_index]
        y_tr = y_train.as_matrix()[train_index]
        x_te = x_train.as_matrix()[test_index]
        y_te = y_train.as_matrix()[test_index]
        
        # PCA
        pca = PCA(n_components=20, random_state=420)
        pca2_results_train = pca.fit_transform(x_tr)
        pca2_results_valid = pca.transform(x_te)
        pca2_results_test = pca.transform(x_test)
        
        x_pca_tr = np.concatenate([pca2_results_train, x_tr], axis=1)
        x_pca_te = np.concatenate([pca2_results_valid, x_te], axis=1)
        x_pca_test = np.concatenate([pca2_results_test, x_test], axis=1)

        clf.train(x_pca_tr, y_tr)

        predictions = clf.predict(x_pca_te)
        score = accuracy_score(y_te, predictions)
        print(score)
        total_accuracy += score
        oof_train[test_index] = predictions
        oof_test_skf[i, :] = clf.predict(x_pca_test)


    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1), total_accuracy / NFOLDS

In [35]:
ada_oof_train, ada_oof_test, total_accuracy = get_oof_with_pca(ada,x_train, y_train, x_test)
print("Adaboost classifier - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.776536312849
0.825842696629
0.808988764045
0.797752808989
0.870786516854
('Adaboost classifier - Total Accuracy with data cleanup and without one hot encoding', 0.81598141987320327)


In [36]:
rf_oof_train, rf_oof_test, total_accuracy = get_oof_with_pca(rf,x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.77094972067
0.780898876404
0.775280898876
0.775280898876
0.820224719101
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.78452702278576358)


Hyperparameter Optimization

In [37]:
# Hyperparameter tuning - Estimators
rf_classifer_optimal =  RandomForestClassifier(n_jobs=-1)
from sklearn.grid_search import RandomizedSearchCV

# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [120, 300, 500, 800, 1200],
           "max_depth" : [6,8,15,25,30],
           "min_samples_leaf" : [1, 2, 5,10],
           "max_features" : ['sqrt', 'log2']
}

CV_rfc = RandomizedSearchCV(estimator=rf_classifer_optimal, param_distributions=param_grid, cv= 3)
CV_rfc.fit(x_train, y_train)

print (CV_rfc.best_params_)

{'n_estimators': 800, 'max_features': 'log2', 'max_depth': 8, 'min_samples_leaf': 1}


In [38]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=CV_rfc.best_params_)
rf_oof_train, rf_oof_test, total_accuracy = get_oof_with_pca(rf,x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.815642458101
0.808988764045
0.831460674157
0.792134831461
0.870786516854
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.82380264892348265)


Model Stacking

In [39]:
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params={})
ada_oof_train, ada_oof_test, total_accuracy = get_oof_with_pca(ada, x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.776536312849
0.825842696629
0.808988764045
0.797752808989
0.870786516854
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.81598141987320327)


In [40]:
gbm = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params={})
gbm_oof_train, gbm_oof_test, total_accuracy = get_oof_with_pca(gbm, x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.821229050279
0.803370786517
0.808988764045
0.780898876404
0.870786516854
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.81705479881991094)


In [41]:
svc = SklearnHelper(clf=SVC, seed=SEED, params={})
svc_oof_train, svc_oof_test, total_accuracy = get_oof_with_pca(svc, x_train, y_train, x_test)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

0.854748603352
0.831460674157
0.814606741573
0.775280898876
0.876404494382
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.830500282468144)


In [42]:
from sklearn.neural_network import MLPClassifier
mlp = SklearnHelper(clf=MLPClassifier, seed=SEED, params={})
mlp_oof_train, mlp_oof_test, overall_accuracy = get_oof_with_pca(mlp, x_train, y_train, x_test)
print("Overall Accuracy", overall_accuracy)

0.810055865922
0.820224719101
0.825842696629
0.780898876404
0.848314606742
('Overall Accuracy', 0.8170673529596385)


In [43]:
x_train_corr = pd.DataFrame( {
     'SVC': svc_oof_train.ravel(), 
     'GBM' : gbm_oof_train.ravel(),
     'RF' : rf_oof_train.ravel(),
     'MLP' : mlp_oof_train.ravel()
})
x_test_corr = pd.DataFrame( {
     'SVC': svc_oof_test.ravel(),
     'GBM' : gbm_oof_test.ravel(),
     'RF' : rf_oof_test.ravel(),
     'MLP' : mlp_oof_test.ravel()
})

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

data = [
    go.Heatmap(
        z= x_train_corr.astype(float).corr().values ,
        x=x_train_corr.columns.values,
        y= x_train_corr.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [44]:
# Hyperparameter tuning - Estimators
rf_classifer_optimal =  RandomForestClassifier(n_jobs=-1)
from sklearn.grid_search import RandomizedSearchCV

# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [120, 300, 500, 800, 1200],
           "max_depth" : [6,8,15,25,30],
           "min_samples_leaf" : [1, 2, 5,10],
           "max_features" : ['sqrt', 'log2']
}

CV_rfc = RandomizedSearchCV(estimator=rf_classifer_optimal, param_distributions=param_grid, cv= 3)
CV_rfc.fit(x_train_corr, y_train)

print (CV_rfc.best_params_)

rf_lev2 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=CV_rfc.best_params_)
rf_lev2_oof_train, rf_lev2_oof_test, total_accuracy = get_oof(rf,x_train_corr, y_train, x_test_corr, False)
print("Random Forest - Total Accuracy with data cleanup and without one hot encoding", total_accuracy)

{'n_estimators': 120, 'max_features': 'sqrt', 'max_depth': 25, 'min_samples_leaf': 5}
0.832402234637
0.842696629213
0.808988764045
0.780898876404
0.876404494382
('Random Forest - Total Accuracy with data cleanup and without one hot encoding', 0.82827819973636296)
